# Rules for knowledge representation

This workbook explores the concept of **rules-based** system in Python.

This workbook guides you into developing a rule-based chatbot. While certain sections are more tutorial-based, others allow you to apply your knowledge independently.

**Make sure to have gone through `W2 - Workbook - Logic.ipynb`** before working on this notebook.


## Chatbots

Chatbots have a long history that dates back to the 1960s. One of the earliest examples is [ELIZA](https://en.wikipedia.org/wiki/ELIZA), a program designed to simulate a conversation with a psychotherapist. ELIZA used pattern matching and simple rules to respond to user inputs. 

Fast forward to the present day, chatbots have become increasingly sophisticated and widely used. They are employed in various industries, including customer service, healthcare, and e-commerce. Rule-based systems are one approach used in chatbot development, where responses are pre-determined based on a set of predefined rules and matches. However, modern chatbots often combine rule-based systems with machine learning techniques.



We are going to develop a mini rule-based chatbot that uses **forward chaining** to identify patterns and generate responses. 

First, we will create a simple chatbot, that analyse user inputs and retrieve relevant rules to form appropriate answers. Then, we will introduce matching rules to generalise the inputs and allow the chatbot to handle a wider range of user queries.

The image below illustrates how the chatbot's inference engine will work:

<figure>
<img src="forward-chaining.png" alt="grid" style="width: 500px;"/>
<figcaption style="text-align:center;font-style:italic">(from Jim Smith's AI module - week 3) </figcaption>    
</figure>
    
In our case,  the long-term memory is the knowledge base, which contains the (hard-coded) collection of information and rules. ; the short-term memory refers to the input provided by the user, which is the immediate context for the chatbot's response.

so, let's get started!

In [ ]:
#!pip install minikanren
import kanren as mk

# we'll keep features from miniKanren in their own mk namespace rather than import them as individual functions.

## Chatbot - basic version

We start with creating a basic version of our bot: the bot receives user inputs, accesses the knowledge base, and generates answers. For this, we need to implement:

- a knowledge base storing the expected inputs as facts (`answers`)
- a function that creates the response from the bot given the user input (`respond`)
- a simple UI where the users can input their questions and receive their answer (`base_chatbot`)

### Knowledge base
Our KB will start small, and we are going to manually code the expected input/answer pairs, using kanren Relation and facts functions.

In [ ]:
# create KB
answers = mk.Relation()

# terms will be of type (expected_input, answer)
# some of the terms are extracted from Jim Smith't AI module - Week 1 : https://github.com/jim-smith/Artificial_Intelligence_1/blob/main/Learning_Materials/week1/myFirstChatbot.aiml
mk.facts(answers, 
         ('HELLO','Well, hello!'),
         ('WHAT ARE YOU', "I'm a bot, isn't clear?"),
         ('IS THIS ALL THERE IS', "I'm afraid so!"),
         ('WHAT IS YOUR NAME', 'My name is UWE bot'),
         ('DO NOT PRETEND YOU ARE HUMAN', "It was worth a try :'("),
         # We'll do something with these later on
         ('WHAT MORE CAN I DO', """The sky is your limit! But if you need some suggestions:{{suggestions}}"""),
         ('WHAT DO YOU LIKE', "I like {{dish}} and {{drink}}")
)

### Getting the right answer

The function below takes the  user input and attempts to find a matching answer in the KB. If an answer is found, it is then returned; otherwise, a default "Sorry, I didn't understand" response is provided.

In [ ]:
def respond(user_input):
    # unknown variable to loop within the KB
    answer = mk.var()

    # find an answer matching the input
    match_answer = mk.run(1, answer, answers(user_input, answer))

    # kanren will return a tuple with some content if it found an answer
    if len(match_answer):
        return match_answer[0]

    return "Sorry, I didn't understand"

Let's check this works OK by printing a couple of responses.

In [ ]:
(
respond('HELLO'),
respond('IS THIS ALL THERE IS'),
respond('CAN YOU DO MY KBHS ASSIGNMENT')
)

### Let's chat!

In the function below, the bot will continuously prompt the user for their input, until they respond with a specific `bye` keyword.

In [ ]:
def base_chatbot():
    keep_chatting = True
    STOP_CHATTING_TRIGGER = 'bye'

    while keep_chatting:
        user_input = input("Enter your message >> ")
        if(user_input == STOP_CHATTING_TRIGGER):
            keep_chatting = False
        else:
            print(respond(user_input))

# Uncomment to run the chatbot
# base_chatbot()

### Exercise 1
In a real-life scenario, a chatbot would typically have a large number of questions and answers, making it impractical to manually encode them all. To handle this, such pairs can be stored in a separate data source.


**Task**

You have been provided with a csv file with some questions/answer.

Write a function that imports this external knowledge source into python and adds it to your KB used by the chatbot.

**File: exercise1.csv**

In [ ]:
# write your code here

## Chatbot - a flexible version

The `respond` rule we have created will match the prompt character for character. Not ideal when it's a human typing things, and  not necessarily uppercasing their input like the prompts are. So let's improve our chatbot to make it more flexible.

We will need to:
- implement a more flexible way to match the prompt to the input, not uppercase/lowercase senstive
- use that new matching rule when searching in the knowledge base

### String transform

In [ ]:
#  transform a string to only its alphanumerical characters and lower case
def transform_string(string):
    # remove non alphanumerical characters, https://stackoverflow.com/a/5843560
    new_str = ''.join(character for character in string if character.isalnum())
    new_str = new_str.lower()
    return new_str

    
# transform two strings and compares them 
# we will use this function in for the rule matching
def string_matching(str1, str2):
    match = (transform_string(str1) == transform_string(str2))
    return match

Quick test:

In [ ]:
(
string_matching('A Bc D', ' a B Cd?'),
string_matching('A c D', 'Something else')
)

### Exercise 2

Our chatbot is able to answer 'Well, hello', whether I type 'HELLO', 'hello', 'Hello!'. 

But what if I type 'helo', 'hlelo'?

Human typing is prone to errors. Users may unintentionally make typos, such as omitting a letter or swapping characters while entering text. 

**Task**

Update the `string_mathcing` function to check for similarities between two texts, rather tahn equality. A useful library could be [textdistance](https://pypi.org/project/textdistance/).

In [ ]:
# write your code here

### Flexible matching

In `respond` we were just accesing the KB and find the exact match for our input. We now need to improve our `respond` function,  by adding a custom goal constructor that matches two string even if they have non-alphanumerical cases and/or lower/upper cases.


As either (or both) of these strings could be logical variables, we'll need use Kanren's `applyo` function: it allows to execute a function of our choice (in our case is `string_matching`), passing it a tuple of arguments that may mix regular python variables and logical variables from Kanren. Once we get the result, we can then run Kanren's `eq` to tell if the goal is fullfilled or not.

In [ ]:
from kanren.term import applyo

# custom matching goal
def alphanumerical_match(str1, str2):
    match = mk.var()

    result = mk.lall(
        # apply the string_matching criteria  
        applyo(string_matching, (str1, str2), match),
        mk.eq(match, True)
    )
    
    return result

We can then combine that new goal constructor with the one matching the `answers`, and create `flexible_response` , an improved version of `respond`.

In [ ]:
def flexible_response(user_input):
    # unknown variables to loop within the KB
    answer = mk.var() 
    expected_input = mk.var()
    
    
    goals = (
        # find candidates for the expeced_input 
        answers(expected_input, answer),
        # check that a candidate matches the user input, after the string transformation
        alphanumerical_match(expected_input, user_input)
    )
    
    # find an answer matching the input
    match_answer = mk.run(1, (answer, expected_input), *goals)
   
   
    # kanren will return a tuple (answer, expected_input) with some content if it found an answer                           
    if len(match_answer):
        return match_answer[0][0]

    return "Sorry, I didn't understand"

Quick test again:

In [ ]:
(
flexible_response('HELLO'),
flexible_response('What are you?'),
flexible_response('is this all there is????')
)

### Improved chatbot

Let's now create a second iteration of the chatbot, a more flexible version.

In [ ]:
def improved_chatbot():
    keep_chatting = True
    STOP_CHATTING_TRIGGER = 'bye'

    while keep_chatting:
        user_input = input("Enter your message >> ")
        
        # use the new way of matching for the 'bye' message as well
        if string_matching(user_input, STOP_CHATTING_TRIGGER):
            keep_chatting = False
        else:
            print(flexible_response(user_input))



# Uncomment to run the chatbot
# improved_chatbot()


### Exercise 3

We want to expand our chatbot features by including user input data, for example their name.

**Task 1**

Write a function so that when the user says `My name is <name>`, then their name gets stored into a variable.

**Task 2**

Make the chatbot to reuse the name.For example, the bot could reply with `Nice to meet you, <name>`, or say `Bye <name>` before exiting the program. 

In [ ]:
# write your code here

## Pulling more content from the knowledge base

You may have noticed in our KB we use `{{suggestions}}` in the answer for 'WHAT MORE CAN I DO'. `{{ }}` is a placeholder to be replaced by some actual suggestions that the bot can answer with. 

**Placeholders** hold some space so they can be replaced with specific values during the execution of the program. This allow to generalise answers, expand the current KB with other knowledge sources, or personalise the chatbot interactions. 

For example, in a real-life scenarios, a chatbot may use placeholders to display information that needs to be repeated across multiple messages (to help with consistency across messages) or that needs to be updated dynamically.




### More knowledge

Let's expand our knowledge base with a list of replacemenents for placeholders.

In [ ]:
# create a new relation
replacements = mk.Relation()

mk.facts(replacements, 
         ('WHAT MORE CAN I DO', (('suggestions', """Can you make the chatbot send multiple answers?
                                                 Can you make the bot ask user name and store it"""))),
         ('WHAT DO YOU LIKE', (('dish','pizza'), ('drink', 'strawberry smoothie')))
   )

Each replacement term lists the expected input it relates to (`'WHAT MORE CAN I DO'`), a key it's meant to replace (`'suggestions'` for our specif placeholder case), and the content to replace it with. 

Let's check everything works.

In [ ]:
x = mk.var()
mk.run(0, x, replacements('WHAT DO YOU LIKE', x))

### Exercise 4

We now want to use these replacements in our chatbot.

**Task 1**

Create a function that queries the KB and outputs both the answer, and any replacements for placeholders. The function should still be flexible in matching the user input with the expected inputs in the KB.

**Task 2**

Using the function from Task 1, combine the answer and its placeholder replacement to form the final output for the user.

**Task 3**

Using the function from Task 2, create a 3rd iteration of the chatbot that will include the replacements in its asnwers.

In [ ]:
# write your code here

## (Optional) Exercises

If you want to keep improving your chatbot, here there are some ideas for you to implement:

- Multiple outputs: Can some inputs make the chatbot send multiple messages?
- Multiple inputs / Incremental decision making: Can the chatbot ask the users multiple questions before reaching a final answer (based on the user inputs)?
- Can you have the bot ask what language the user would prefer to talk in and change the text of its answers according to their preference?